# Poceni Pythonska Pojedina
## Seminarska naloga


### Pridobivanje podatkov
Podatke pridobljene s pomo[ch]jo `mercator_scraper.py` in pa `okusno_scraper.py` najprej uvozimo za uporabo pri analizi.

In [5]:
import pandas as pd 
import numpy as np

recepti = pd.read_csv("data/recepti.csv")
recepti.head()

,Avtorji,Tezavnost,Cas priprave,Cas kuhanja,Skupen cas,Dolzina navodil,Energijska vrednost,Beljakovine,Ogljikovi hidrati,Mascobe,Vlaknine,Vitamini
0,M.J.,1,15,40,55,1276,315.98,1.80,11.00,2.21,1.36,0.00
1,Su.S.,1,20,40,60,1290,1260.01,6.38,60.20,2.67,0.74,0.30
2,Sanja Sirk,1,15,35,50,1525,579.02,15.07,2.62,6.46,0.09,0.00
3,M. J.,1,10,30,40,1116,145.44,0.16,3.97,1.59,0.24,0.00
4,M. J.,1,20,90,110,2125,654.54,7.66,9.00,9.33,1.02,0.04


[Zh]e samo iz te tabele lahko izklu[sh]imo prevec veliko informacij, recimo kak[sh]na je odvisnost med [ch]asom kuhanja in pa energijsko vrednostjo jedi? Kaj pa med energijsko vrednostjo ter vsebnostjo raznoraznih hranilnih snovi, npr. ali imajo kalori[ch]no bogatej[sh]e jedi ve[ch] ma[sh][ch]ob? Kaj pa ogljikovih hidratov? Pa poglejmo.